# Reductions

ironArray supports a broad range of reduction facilities, like `sum`, `min`, `max`, `mean` and others.  Also, they work on any (or group of) dimensions.  One interesting aspect of these is that the implementation leverages the multi-threading capabilities of ironArray, so they can be pretty fast (although some times they need some help from the user).

In order to exercise some of this functionality, let's download some open data from the network.  In this case, we are interested in downloading precipitation data from a period of 3 months.  With a decent connection, this should not take more than a minute to download.  Let's go:

In [1]:
import xarray as xr
import numpy as np
import s3fs
import iarray as ia

In [2]:
def open_zarr(year, month, datestart, dateend):
    fs = s3fs.S3FileSystem(anon=True)

    datestring = 'era5-pds/zarr/{year}/{month:02d}/data/'.format(year=year, month=month)

    precip_zarr = xr.open_zarr(s3fs.S3Map(datestring + 'precipitation_amount_1hour_Accumulation.zarr/', s3=fs))
    precip_zarr = precip_zarr.sel(time1=slice(np.datetime64(datestart), np.datetime64(dateend)))

    return precip_zarr.precipitation_amount_1hour_Accumulation

In [3]:
%%time

precip_m1 = open_zarr(1987, 10, '1987-10-01', '1987-10-30 23:59')
precip_m2 = open_zarr(1987, 11, '1987-11-01', '1987-11-30 23:59')
precip_m3 = open_zarr(1987, 12, '1987-12-01', '1987-12-30 23:59')

CPU times: user 978 ms, sys: 71.8 ms, total: 1.05 s
Wall time: 39.5 s


Let's see how one of these arrays of monthly precipitation looks like:

In [4]:
precip_m1

<xarray.DataArray 'precipitation_amount_1hour_Accumulation' (time1: 720, lat: 721, lon: 1440)>
dask.array<getitem, shape=(720, 721, 1440), dtype=float32, chunksize=(372, 150, 150), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time1    (time1) datetime64[ns] 1987-10-01 ... 1987-10-30T23:00:00
Attributes:
    long_name:       Total precipitation
    nameCDM:         Total_precipitation_1hour_Accumulation
    nameECMWF:       Total precipitation
    product_type:    forecast
    shortNameECMWF:  tp
    standard_name:   precipitation_amount
    units:           m

Ok, so that's about 3 GB per month, so the full dataset is about 9 GB.  Now, let's build a NumPy array out of this (be careful if you plan to run this in your local machine, as you will need at least 12 GB of free memory):

In [5]:
%%time

precip = np.stack((precip_m1.values, precip_m2.values, precip_m3.values))
precip.shape

CPU times: user 18.5 s, sys: 5.68 s, total: 24.2 s
Wall time: 41.8 s


(3, 720, 721, 1440)

Now, let's suppose that we want to compute the mean for the precipitation per hour during our period of time.  As the time axis is 1 in our new `precip` array, we want to reduce on all axis, except the first.  That is:

In [6]:
%%time

reduc0 = np.mean(precip, axis=(0, 2, 3))

CPU times: user 1.26 s, sys: 204 ms, total: 1.46 s
Wall time: 1.46 s


Ok.  Now, let's import this data into ironArray before proceeding with reductions:

In [7]:
%%time

ia_precip = ia.numpy2iarray(precip)
print(ia_precip)
print(ia_precip.cratio)

<IArray (3, 720, 721, 1440) np.float32>
9.711491423121984
CPU times: user 14.1 s, sys: 1.62 s, total: 15.7 s
Wall time: 8.95 s


Ok, so ironArray achieves a compression ratio of almost 10x, which is a big win in terms of memory consumption.  Now, let's have a look at how reduction works:

In [8]:
%%time
reduc1 = ia.mean(ia_precip, axis=(0, 2, 3))

CPU times: user 6min 29s, sys: 3.26 s, total: 6min 32s
Wall time: 39.2 s


Ok, so that's pretty slow.  Now, it is time to remember that ironArray uses chunked storage, even when it holds data in-memory.  In this case, we have been traversing the array in a very innefficient way.  In general, in chunked storage, it is always better to start reducing by the dimension that is the largest, and we took the inverse order.  With this in mind, let's try with a more reasonable order:

In [9]:
%%time
reduc1 = ia.mean(ia_precip, axis=(3, 2, 0)).data

CPU times: user 9.17 s, sys: 415 ms, total: 9.59 s
Wall time: 1.24 s


Ok, that's much better.  This time is pretty competitive.  Now, let's compare actual data with NumPy (just in case):

In [10]:
np.testing.assert_almost_equal(reduc0, reduc1)

Ok, so the mean calculation has worked correctly.

If you need to compute this sort of reductions lots of times, you may want to fine tune your ironArray array parameters.  If so, just keep reading.  But before proceeding further, let's save our data for later reload it:

In [11]:
%%time

ia.save(ia_precip, "precip-3m.iarr")
%ls -lh precip-3m.iarr

-rw-r--r--  1 faltet  staff   1.0G Dec  4 14:56 precip-3m.iarr
CPU times: user 71.6 ms, sys: 1.06 s, total: 1.13 s
Wall time: 1.64 s


The dataset is stored now on a single file of 1 GB, which is almost 10x less that the original dataset thanks to compression, and ready to be used later on.

## Optimization Tips

As we know, most of ironArray optimization stems from their two levels of partitioning.  Previously, we have been using automatic values for chunkshape and blockshape (based on CPU's cache sizes).  But for maximum speed, there is no replacement for fine tuning chunk and block shapes manually.

Let's start by loading the array that we previously stored on-disk:

In [12]:
%%time
import iarray as ia
ia_precip = ia.load("precip-3m.iarr")

CPU times: user 4.74 ms, sys: 93.7 ms, total: 98.4 ms
Wall time: 97.2 ms


So, not too much for loading a 9 GB large array from disk, uh?  Well, the thing is that `load()` just loads data when it needs it by default (but see its `load_in_mem` argument).  So, only a tiny portion of the file (the metadata) is read in order to figure out how access the data.

Now, let's see the current chunk and block shapes:

In [13]:
print(ia_precip.chunkshape)
print(ia_precip.blockshape)

(1, 64, 64, 128)
(1, 16, 16, 64)


Now, let's start with a first attempt to find out the chunk and block shapes that can optimize the reductions.  Initially one may think that making the last dimension (the largest) as large as possible could be good decision.  Let's try that by creating another container:

In [14]:
%%time
with ia.config(chunkshape=(1, 64, 64, 1440), blockshape=(1, 16, 16, 360)) as cfg:
    new_precip = ia_precip.copy(cfg=cfg)

CPU times: user 19.1 s, sys: 4.5 s, total: 23.6 s
Wall time: 12.2 s


Ok, let's try the reduction with the new array:

In [15]:
%%time
reduc1 = ia.mean(new_precip, axis=(3, 2, 0)).data

CPU times: user 6.55 s, sys: 669 ms, total: 7.22 s
Wall time: 981 ms


Well, we did get some improvement, which is not bad for a first attempt.  After several iterations, one can arrive a sort of optimal configuration:

In [16]:
%%time
with ia.config(chunkshape=(1, 360, 128, 1440), blockshape=(1, 8, 8, 720)) as cfg:
    new_precip = ia_precip.copy(cfg=cfg)


CPU times: user 16.5 s, sys: 2.89 s, total: 19.4 s
Wall time: 12.1 s


In [17]:
%%time
reduc1 = ia.mean(new_precip, axis=(3, 2, 0)).data

CPU times: user 6.55 s, sys: 76.4 ms, total: 6.62 s
Wall time: 608 ms


So, this new time is competitive with NumPy (a bit better, in fact).  That means that, when you have to deal with large arrays you should not assume that working with NumPy arrays is the fastest thing on earth.  With the right tools, and a bit of manual tuning, you can end with very fast solutions for this scenario too.

But we have tested just in-memory ironArrays.  What's up with on-disk arrays?  Let's create one and use it for reductions:

In [18]:
%%time
ia.save(new_precip, "precip-3m-optimal.iarr")


CPU times: user 58.3 ms, sys: 903 ms, total: 961 ms
Wall time: 986 ms


and let's re-open again but in 'lazy' mode and do the reduction from disk:

In [19]:
import iarray as ia

In [20]:
new_precip_opt = ia.load("precip-3m-optimal.iarr")

In [21]:
%%time
## reduc1 = ia.mean(new_precip_opt, axis=(3, 2, 0)).data


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


TODO: the about 'lazy' reduce does not never finish...